<img src="images/dask_horizontal.svg" align="right" width="30%">

# Table of Contents
* [Bag: Parallel Lists for semi-structured data](#Bag:-Parallel-Lists-for-semi-structured-data)
	* [Creation](#Creation)
	* [Manipulation](#Manipulation)
		* [Exercise: Accounts JSON data](#Exercise:-Accounts-JSON-data)
		* [Basic Queries](#Basic-Queries)
		* [Use `flatten` to de-nest](#Use-flatten-to-de-nest)
		* [Groupby and Foldby](#Groupby-and-Foldby)
		* [`groupby`](#groupby)
		* [`foldby`](#foldby)
		* [Example with account data](#Example-with-account-data)
		* [Exercise: compute total amount per name](#Exercise:-compute-total-amount-per-name)
	* [DataFrames](#DataFrames)
		* [Denormalization](#Denormalization)
	* [Limitations](#Limitations)


# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

*  [Bag Documenation](http://dask.pydata.org/en/latest/bag.html)
*  [Bag API](http://dask.pydata.org/en/latest/bag-api.html)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc..

In [94]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [95]:
# each element is a text file of JSON lines
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))

In [96]:
# Requires `s3fs` library
# each element is a remote CSV text file
b = db.read_text('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv')

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

As with `Array` and `DataFrame` objects, operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution.  

Dask.bag uses `dask.multiprocessing.get` by default.  For examples using dask.bag with a distributed cluster see 

*  [Dask.distributed docs](http://distributed.readthedocs.io/en/latest/)
*  [Blogpost analyzing github data on EC2](https://www.continuum.io/content/dask-distributed-and-anaconda-cluster)

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<map-lam..., npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Exercise: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Ingrid", "transactions": [{"transaction-id": 113, "amount": 596}, {"transaction-id": 147, "amount": 600}, {"transaction-id": 207, "amount": 641}, {"transaction-id": 220, "amount": 672}, {"transaction-id": 315, "amount": 625}, {"transaction-id": 323, "amount": 654}, {"transaction-id": 752, "amount": 618}, {"transaction-id": 1023, "amount": 614}, {"transaction-id": 1112, "amount": 600}, {"transaction-id": 1142, "amount": 648}, {"transaction-id": 1187, "amount": 620}, {"transaction-id": 1554, "amount": 554}, {"transaction-id": 1744, "amount": 660}, {"transaction-id": 2014, "amount": 628}, {"transaction-id": 2063, "amount": 596}, {"transaction-id": 2290, "amount": 666}, {"transaction-id": 2319, "amount": 639}, {"transaction-id": 2371, "amount": 647}, {"transaction-id": 2375, "amount": 647}, {"transaction-id": 2413, "amount": 627}, {"transaction-id": 2775, "amount": 622}, {"transaction-id": 2865, "amount": 655}, {"transaction-id": 2986, "amount": 570}, {"transaction-id"

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Ingrid',
  'transactions': [{'amount': 596, 'transaction-id': 113},
   {'amount': 600, 'transaction-id': 147},
   {'amount': 641, 'transaction-id': 207},
   {'amount': 672, 'transaction-id': 220},
   {'amount': 625, 'transaction-id': 315},
   {'amount': 654, 'transaction-id': 323},
   {'amount': 618, 'transaction-id': 752},
   {'amount': 614, 'transaction-id': 1023},
   {'amount': 600, 'transaction-id': 1112},
   {'amount': 648, 'transaction-id': 1142},
   {'amount': 620, 'transaction-id': 1187},
   {'amount': 554, 'transaction-id': 1554},
   {'amount': 660, 'transaction-id': 1744},
   {'amount': 628, 'transaction-id': 2014},
   {'amount': 596, 'transaction-id': 2063},
   {'amount': 666, 'transaction-id': 2290},
   {'amount': 639, 'transaction-id': 2319},
   {'amount': 647, 'transaction-id': 2371},
   {'amount': 647, 'transaction-id': 2375},
   {'amount': 627, 'transaction-id': 2413},
   {'amount': 622, 'transaction-id': 2775},
   {'amount': 655, 'transaction-id':

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 4,
  'name': 'Alice',
  'transactions': [{'amount': 2165, 'transaction-id': 23644},
   {'amount': 2059, 'transaction-id': 26639},
   {'amount': 2079, 'transaction-id': 27865},
   {'amount': 2290, 'transaction-id': 29172},
   {'amount': 2083, 'transaction-id': 42535},
   {'amount': 1966, 'transaction-id': 42873},
   {'amount': 2056, 'transaction-id': 55610},
   {'amount': 2235, 'transaction-id': 78577},
   {'amount': 2039, 'transaction-id': 85934},
   {'amount': 2106, 'transaction-id': 86125},
   {'amount': 1981, 'transaction-id': 92607},
   {'amount': 2056, 'transaction-id': 94773}]},
 {'id': 59,
  'name': 'Alice',
  'transactions': [{'amount': -439, 'transaction-id': 1424},
   {'amount': -426, 'transaction-id': 4712},
   {'amount': -418, 'transaction-id': 7901},
   {'amount': -493, 'transaction-id': 13175},
   {'amount': -486, 'transaction-id': 27323},
   {'amount': -323, 'transaction-id': 35246},
   {'amount': -404, 'transaction-id': 52277},
   {'amount': -407, 'transaction-i

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'count': 12, 'name': 'Alice'},
 {'count': 13, 'name': 'Alice'},
 {'count': 3, 'name': 'Alice'},
 {'count': 130, 'name': 'Alice'},
 {'count': 39, 'name': 'Alice'})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(12, 13, 3, 130, 39)

In [14]:
# Average number of transactions for all of the Alice's
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

117.54334226988382

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'amount': 2165, 'transaction-id': 23644},
  {'amount': 2059, 'transaction-id': 26639},
  {'amount': 2079, 'transaction-id': 27865},
  {'amount': 2290, 'transaction-id': 29172},
  {'amount': 2083, 'transaction-id': 42535},
  {'amount': 1966, 'transaction-id': 42873},
  {'amount': 2056, 'transaction-id': 55610},
  {'amount': 2235, 'transaction-id': 78577},
  {'amount': 2039, 'transaction-id': 85934},
  {'amount': 2106, 'transaction-id': 86125},
  {'amount': 1981, 'transaction-id': 92607},
  {'amount': 2056, 'transaction-id': 94773}],
 [{'amount': -439, 'transaction-id': 1424},
  {'amount': -426, 'transaction-id': 4712},
  {'amount': -418, 'transaction-id': 7901},
  {'amount': -493, 'transaction-id': 13175},
  {'amount': -486, 'transaction-id': 27323},
  {'amount': -323, 'transaction-id': 35246},
  {'amount': -404, 'transaction-id': 52277},
  {'amount': -407, 'transaction-id': 56290},
  {'amount': -282, 'transaction-id': 87236},
  {'amount': -402, 'transaction-id': 87325},
  {'amount':

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'amount': 2165, 'transaction-id': 23644},
 {'amount': 2059, 'transaction-id': 26639},
 {'amount': 2079, 'transaction-id': 27865})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(2165, 2059, 2079)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

671.9832130828474

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

KeyboardInterrupt: 

In [81]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, smt):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 1119), ('Bob', 900), ('Charlie', 619), ('Dan', 1450), ('Edith', 641), ('Frank', 700), ('George', 1077), ('Hannah', 948), ('Ingrid', 916), ('Jerry', 1350), ('Kevin', 1150), ('Laura', 1149), ('Michael', 1450), ('Norbert', 700), ('Oliver', 650), ('Patricia', 848), ('Quinn', 1300), ('Ray', 900), ('Sarah', 893), ('Tim', 1300), ('Ursula', 1050), ('Victor', 750), ('Wendy', 800), ('Xavier', 700), ('Yvonne', 550), ('Zelda', 950)]
CPU times: user 60 ms, sys: 0 ns, total: 60 ms
Wall time: 3.27 s


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [104]:
from operator import add

def sum_amounts(i, z):
    return i + sum([y["amount"] for y in z["transactions"]])

result = js.foldby(key='name', 
                   binop=sum_amounts, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()

print(sorted(result))

[('Alice', 88386624), ('Bob', 155862818), ('Charlie', 46119749), ('Dan', 390689784), ('Edith', 184112890), ('Frank', 118948950), ('George', 314619696), ('Hannah', 75245797), ('Ingrid', 201673179), ('Jerry', 565487741), ('Kevin', 139911058), ('Laura', 145569481), ('Michael', 486405087), ('Norbert', 150549445), ('Oliver', 224242162), ('Patricia', 183873996), ('Quinn', 267614229), ('Ray', 75129801), ('Sarah', 125422960), ('Tim', 300737659), ('Ursula', 108107588), ('Victor', 123373029), ('Wendy', 109154942), ('Xavier', 92426274), ('Yvonne', 116395300), ('Zelda', 205386397)]


## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [83]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Ingrid,"[{'transaction-id': 113, 'amount': 596}, {'tra..."
1,1,Michael,"[{'transaction-id': 160, 'amount': -60}, {'tra..."
2,2,Wendy,"[{'transaction-id': 303, 'amount': 1754}, {'tr..."
3,3,Ray,"[{'transaction-id': 2318, 'amount': 5148}, {'t..."
4,4,Alice,"[{'transaction-id': 23644, 'amount': 2165}, {'..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [84]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 3.39 s, sys: 0 ns, total: 3.39 s
Wall time: 4.27 s


name
Alice      1119
Bob         900
Charlie     619
Dan        1450
Edith       641
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [85]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'amount': 596, 'id': 0, 'name': 'Ingrid', 'transaction-id': 113},
 {'amount': 600, 'id': 0, 'name': 'Ingrid', 'transaction-id': 147},
 {'amount': 641, 'id': 0, 'name': 'Ingrid', 'transaction-id': 207})

In [86]:
df = transactions.to_dataframe()
df.head()

,amount,id,name,transaction-id
0,596,0,Ingrid,113
1,600,0,Ingrid,147
2,641,0,Ingrid,207
3,672,0,Ingrid,220
4,625,0,Ingrid,315


In [87]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 6.4 s, sys: 0 ns, total: 6.4 s
Wall time: 6.78 s


name
Alice       131531
Bob         205440
Charlie     166890
Dan         273003
Edith       121074
Frank       136151
George      178006
Hannah      152562
Ingrid      206696
Jerry       310527
Kevin       188934
Laura       202346
Michael     367747
Norbert     136312
Oliver      123540
Patricia    208075
Quinn       218862
Ray         190726
Sarah       161787
Tim         373222
Ursula      162866
Victor      191689
Wendy       134730
Xavier      158489
Yvonne      119728
Zelda       179067
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.